In [1]:
# !sudo apt-get install python3.7
# !sudo apt-get update -y
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
# !sudo update-alternatives --config python3

In [1]:
!pip install numpy
!pip install pyopenms
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 MB 4.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 8.1 MB/s eta 0:00:00a 0:00:01


In [3]:
import re

EXPERIMENT_NAME = "OVFF364"
# reading file
file1 = open(f'./{EXPERIMENT_NAME}.txt', 'r')
oligo_list = [line for line in file1.readlines() if re.search("^[a-zA-Z][1-9]", line)]
import csv

header = ['where\tcode\tseq\tDMT\tcomment\tlength\t']
data = [row.strip() for row in oligo_list]


with open('./oligos.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)

    writer = csv.writer(f, quotechar='\"', delimiter='\n', quoting=csv.QUOTE_NONE, escapechar='\\')
    # write the data
    writer.writerow(data)

In [4]:
import pandas as pd

df = pd.read_csv('./oligos.csv', delimiter='\t')
# drop NaN seq
df = df[df['seq'].notna()]
oligo_list = df['seq'].to_list()

In [5]:
from Bio.Seq import Seq
from Bio.SeqUtils import molecular_weight

# Bio.SeqUtils.molecular_weight(seq, seq_type='DNA', double_stranded=False, circular=False, monoisotopic=False)

from pyopenms import *

# # reading file
# file1 = open('./OVFF317_oligos_seq.txt', 'r')
# oligo_list = file1.readlines()

mass_list=[]
# https://biopython.org/docs/1.75/api/Bio.SeqUtils.html
for oligo_seq in oligo_list: 
  try:
    my_dna = Seq(oligo_seq)
  except:
    print(oligo_seq)
  try:
    print(f"{oligo_seq.strip()}\t" + "%0.2f" % (molecular_weight(my_dna)-79.97))
    mass_list+=["%0.2f" % (molecular_weight(my_dna)-79.97)]
  except ValueError:
    print(f"{oligo_seq.strip()}\t" + "'N' is not a valid unambiguous letter for DNA")
    mass_list+=["NaN"]

TAT AGG ATC CAT GAA GGT CGA TGA GTT GC	8996.82
TAT AGT CGA CGA TTT ACC CCG GCA GAC	8244.34
TAT AGG ATC CTG TCT GCC GGG GTA AA	8026.19
TAT AGT CGA CAC GTA AGA AAT CTT CAA GGG TT	9871.41
TAT AGG ATC CAT GGG AAG CTC CCA CC	7940.15
TAT AGT CGA CAG GGG AGT AGT CAG ACC	8373.43
TAT AGG ATC CGG GTC TGA CTA CTC CCC	8211.30
TAT AGT CGA CAT AGT CGT TGA TCG TCT TCT G	9507.13
GAT TTA CCC CGG CAG AC	5155.37
TGT CTG CCG GGG TAA A	4937.22
AGG GGA GTA GTC AGA CC	5284.45
GGG TCT GAC TAC TCC CC	5122.33
CAT TAG TTT TCG TTA ACA CGC G	6700.36
GAT CGA TAT AAA GTT GGG AAG TAC G	7794.07
GCG GTA AAG AAA ATT TTG AGC G	6847.47
TGG GTC GAT ATA AAG CTT GCT CA	7078.60
ATG AAG GTC GAT GAG TTG CCG GTC GAT GAA CGT TTG AAG GCA GTG TT	14669.42
GGT GGA TAA AGC TCC TCA ATT CCG CGC TCC TTC AAC ACT GCC TTC AAA CGT TCA T	16740.75
GGA ATT GAG GAG CTT TAT CCA CCA CAG GCT GAA GCT TTA AAA TCC GG	14511.35
CTA AGT TGC GCC CCT CCA ATG CTC CGG ATT TTA AAG CTT CAG CCT	13682.79
TTG GAG GGG CGC AAC TTA GTT CTG GCG ATT CCT ACG GCT TCC GG

In [6]:
df['monoisotopic_mass']=mass_list

In [7]:
export_dataframe=df[['where', 'code', 'length', 'seq', 'monoisotopic_mass']]

In [8]:
export_dataframe.to_csv(f'./{EXPERIMENT_NAME}_calculated.csv', sep = '\t', index=False)

In [9]:
export_dataframe


,where,code,length,seq,monoisotopic_mass
0,A1,p03_F1-Fb,#= 29,TAT AGG ATC CAT GAA GGT CGA TGA GTT GC,8996.82
1,B1,p03_F1-Rs,#= 27,TAT AGT CGA CGA TTT ACC CCG GCA GAC,8244.34
2,C1,p03_F2-Fb,#= 26,TAT AGG ATC CTG TCT GCC GGG GTA AA,8026.19
3,D1,p03_F2-Rs,#= 32,TAT AGT CGA CAC GTA AGA AAT CTT CAA GGG TT,9871.41
4,E1,p10_F1-Fb,#= 26,TAT AGG ATC CAT GGG AAG CTC CCA CC,7940.15
...,...,...,...,...,...
359,H23,unitail-oxford,#= 27,TTA ACC TAg TCA ggC AgC AAA AgC Agg,8326.42
360,I23,InfA_-BC01/2R,#= 59,GGT GCT GAA GAA AGT TGT CGG TGT CTT TGT GTT AA...,18457.87
361,J23,Runitail-oxford,#= 28,TTA ACC TAg AAA ggg AgT AgA AAC AAg g,8727.70
362,K23,InfA_-BC02/2F,#= 58,GGT GCT GTC GAT TCC GTT TGT AGT CGT CTG TTT AA...,17934.49


In [10]:
molecular_weight(Seq("TG"))

651.4144

In [11]:
print([ord(char) - 96 for char in input('Write Text: ').lower()])

Write Text: 
[]


In [12]:
seq = "CCCGGGGTACCACGGAGAGATGGTGGAAATCATCTTTCTCGTGGGCATCCTTGATGGCCACCTCGTCGGAAGTGCCCACGAGGATGACAGCAATGCCAATGCTGGGGGGGCTCTTCTGAGAACGAGCTCTGCTGCCTGACACGGCCAGGACGGCCAACACCAACCAGAACTTGGGAGAACAGCACTCCGCTCTGGGCTTCATCTTCAACTCGTCGACTCCCTGCAAACACAAAGAAAGAGCATGTTAAAATAGGATCTACATCACGTAACCTGTCTTAGAAGAGGCTAGATACTGCAATTCAAGGACCTTATCTCCTTTCATTGAGCACCAAACCCAACTCCATCTACCAGCCTACTCTCTTATCTCTGGTATT"


In [13]:
seq.upper()

'CCCGGGGTACCACGGAGAGATGGTGGAAATCATCTTTCTCGTGGGCATCCTTGATGGCCACCTCGTCGGAAGTGCCCACGAGGATGACAGCAATGCCAATGCTGGGGGGGCTCTTCTGAGAACGAGCTCTGCTGCCTGACACGGCCAGGACGGCCAACACCAACCAGAACTTGGGAGAACAGCACTCCGCTCTGGGCTTCATCTTCAACTCGTCGACTCCCTGCAAACACAAAGAAAGAGCATGTTAAAATAGGATCTACATCACGTAACCTGTCTTAGAAGAGGCTAGATACTGCAATTCAAGGACCTTATCTCCTTTCATTGAGCACCAAACCCAACTCCATCTACCAGCCTACTCTCTTATCTCTGGTATT'

In [ ]:
len(seq)
my_dna = Seq(seq)
molecular_weight(my_dna)